# How do PV infeed change phasor angle information?

#### Libraries

In [ ]:
#Importing packages that may be needed:

import pandas as pd # data processing
import numpy as np # linear algebra
import math
import dask.dataframe as dd
import os
from tqdm import tqdm

import statsmodels.api as sm
#import statsmodels.api as sm

import matplotlib.pyplot as plt # this is used for the plot the graph
import plotly.express as px # this is used for the plot the graph
import seaborn as sns # used for plot interactive graph.
from matplotlib import pylab

from sklearn.metrics import r2_score

import sys 
from scipy.stats import randint
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.model_selection import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

## Energy solar data

In [ ]:
path1 = '../input/pv-data/solar-15m_2019-09-01_2020-09-30.csv'
PV_df = pd.read_csv(path1)

In [ ]:
PV_df.head()

In [ ]:
PV_df.info()

In [ ]:
PV_df.shape

In [ ]:
display(PV_df.head())
display(PV_df.tail())

In [ ]:
# Number of missing values in each column of dataframe
missing_val = (PV_df.isna().sum())
print("missing values in each column: \n",missing_val)

In [ ]:
PV_df['time'] = pd.to_datetime(PV_df['time'], format='%Y-%m-%d %H:%M')

PV_df['Date'] = [d.date() for d in PV_df['time']]
PV_df['Time'] = [d.time() for d in PV_df['time']]

PV_df = PV_df.set_index(['time'])
PV_df.head()

In [ ]:
# start and end dates of the time series
print ("solar energy dataset: start_date = {}, end_date = {} \n".format(PV_df.index.min(), PV_df.index.max()))

In [ ]:
%%time
PV_df.plot(grid =True, subplots=True, figsize = (20, 20))
plt.xlabel('time', fontsize=16)
plt.legend(loc="upper left")
plt.tight_layout()

In [ ]:
PV_df.describe()

In [ ]:
PV_df = PV_df.reset_index(['time'])
#aggregate_pv_df = PV_df.copy()[['Date', 'Time']]
aggregate_pv_df = PV_df.copy()[['time']]
aggregate_pv_df['Hochrechnung_Total'] =  PV_df[['Solarenergie_Hochrechnung_50Hertz','Solarenergie_Hochrechnung_Amprion', 'Solarenergie_Hochrechnung_TenneT_TSO', 'Solarenergie_Hochrechnung_TransnetBW']].sum(axis=1)
aggregate_pv_df['Prognose_Total'] =  PV_df[['Solarenergie_Prognose_50Hertz','Solarenergie_Prognose_Amprion', 'Solarenergie_Prognose_TenneT_TSO', 'Solarenergie_Prognose_TransnetBW']].sum(axis=1)

aggregate_pv_df.head()

In [ ]:
aggregate_pv_df = aggregate_pv_df.set_index(['time'])
aggregate_pv_df.head()

In [ ]:
# start and end dates of the time series
print ("PV dataset: start_date = {}, end_date = {} \n".format(aggregate_pv_df.index.min(), aggregate_pv_df.index.max()))

In [ ]:
aggregate_pv_df.shape

In [ ]:
aggregate_pv_df.describe()

In [ ]:
%%time
aggregate_pv_df['Hochrechnung_Total'].plot(grid = True, figsize = (16, 9))
aggregate_pv_df['Prognose_Total'].plot(grid = True, figsize = (16, 9))
plt.xlabel('time', fontsize=16)
plt.legend(loc="upper left")
plt.tight_layout()

### Heatmap visualization

In [ ]:
pv_heatmap_df = aggregate_pv_df.copy()
pv_heatmap_df = pv_heatmap_df.reset_index()

time_series_2020 = (pv_heatmap_df['time'] >= ('2019-09-01')) & (pv_heatmap_df['time'] <= ('2020-09-30'))
from_jan_2020 = pv_heatmap_df.loc[time_series_2020]
from_jan_2020 = from_jan_2020.set_index(['time'])

#Create variables for Day and hour
from_jan_2020['day'] = [i.dayofyear for i in from_jan_2020.index]
from_jan_2020['hour'] = [i.hour for i in from_jan_2020.index]

# group by month and year, get the average
from_jan_2020 = from_jan_2020.groupby(['day', 'hour']).mean()

from_jan_2020= from_jan_2020['Hochrechnung_Total'].unstack(level=0)

fig, ax = plt.subplots(figsize=(25, 10))
#cmap = "Reds"
#cmap = "RdPu"
#cmap = "YlOrRd"
#cmap = "autumn"
#cmap = "hot"
#cmap="OrRd"
#cmap = "Oranges"
sns.heatmap(from_jan_2020, vmin=0)
plt.xlabel("Day", fontsize=14)
plt.ylabel("Hour of the Day", fontsize=14)

# This sets the yticks "upright" with 0, as opposed to sideways with 90.
plt.yticks(rotation=0)
#plt.xticks(rotation=0)
plt.show()

In [ ]:
pv_heatmap_df2 = aggregate_pv_df.copy()
pv_heatmap_df2 = pv_heatmap_df2.reset_index()

time_series_2020 = (pv_heatmap_df2['time'] >= ('2020-01-01')) & (pv_heatmap_df2['time'] <= ('2020-09-30'))
from_jan_2020 = pv_heatmap_df2.loc[time_series_2020]
from_jan_2020 = from_jan_2020.set_index(['time'])

#Create variables for Day and hour
from_jan_2020['month'] = [i.month for i in from_jan_2020.index]
from_jan_2020['hour'] = [i.hour for i in from_jan_2020.index]

# group by month and year, get the average
from_jan_2020 = from_jan_2020.groupby(['month', 'hour']).mean()
from_jan_2020 = from_jan_2020['Hochrechnung_Total'].unstack(level=0)

fig, ax = plt.subplots(figsize=(10, 5))

#cmap = "Reds"
#cmap = "RdPu"
#cmap="OrRd"
#cmap = "Oranges"

#import cmocean
#cmap = cmocean.cm.oxy
#plt.contourf(from_jan_2020, 20, cmap=cmap)

sns.heatmap(from_jan_2020, vmin=0)
plt.title('Year 2020', fontsize=14, fontweight='bold')
plt.xlabel("Month_Number", fontsize=14)
plt.ylabel("Hour of the Day", fontsize=14)
plt.show()

#### Frequency Time Series Decomposition

The frequency of decomposition must be an interval, which 'may' repeat. So we have data with 15min frequency and we are looking for a weekly repetition behavior.
To look for a weekly repetition behavior, let's use:
$decompfreq = \cfrac{24h \cdot 60min}{15min} \cdot 7days$

In [ ]:
decompfreq = int(((24*60)/15)*7)
res = sm.tsa.seasonal_decompose(aggregate_pv_df.Hochrechnung_Total.interpolate(),
                               period=decompfreq,model='additive')
pylab.rcParams['figure.figsize'] = (14, 9)
resplot = res.plot()

# Phasor angle data

### Data Pre-processing

In [ ]:
path2 = '../input/phasor-data/phase-angle-1s_2019-09-01_2020-09-30.csv'

In [ ]:
# Peep at the training file header
df_tmp = pd.read_csv(path2, nrows=901, parse_dates =['time'], keep_date_col = True)
df_tmp

In [ ]:
df_tmp.info()

In [ ]:
missing_val = (df_tmp.isna().sum())
print(missing_val)

In [ ]:
#Let's take locations Herzogenrath and Schondorf for demonstration purpose
df_tmp = df_tmp.set_index(['time'])  

df_tmp['phase_diff'] = df_tmp['Herzogenrath'] - df_tmp['Schondorf']
df_tmp['angle'] = np.sin(np.deg2rad(df_tmp['phase_diff']))

### Visualizing 15 min interval of the time series

In [ ]:
fig = px.line(x=df_tmp.index, y=df_tmp['Herzogenrath'], title='site1 \n in degrees')
fig.update_xaxes()
fig.show()

In [ ]:
fig = px.line(x=df_tmp.index, y=df_tmp['phase_diff'], title='site1 - site2 \n  in degrees')
fig.update_xaxes()
fig.show()

In [ ]:
fig = px.line(x=df_tmp.index, y=df_tmp['angle'], title='sin (site1 - site2 \n)')
fig.update_xaxes()
fig.show()

In [ ]:
#Locations
site1 = "Winterthur"
site2 = "Büdingen"
site3 = "Schondorf"
site4 = "Herzogenrath"
site5 = "Bremen"
site6=  "Dresden"
site7 = "Lleida"
site8 = "Sibiu"
site9 = "Belfort Cedex"
site10 = "Wien (SBA)"

In [ ]:
chunksize = 10000000

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(path2, usecols=("time",site1, site2,site3,site4,site5,site6,site7,site8,site9,site10),parse_dates =['time'], keep_date_col = True, chunksize=chunksize)):
     
    df_chunk = df_chunk.set_index(['time'])
    #df_chunk['time'] = pd.to_datetime(df_chunk['time'], utc=True, format='%Y-%m-%d %H:%M:%S')
    
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk)
    

# Merge all dataframes into one dataframe
data = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
data.info()

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
# start and end dates of the time series
print ("phase angle dataset: start_date = {}, end_date = {} \n".format(data.index.min(), data.index.max()))

In [ ]:
missing_val = (data.isna().sum())
print(missing_val)

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(path2, usecols=("time",site1, site2,site3,site4,site5,site6,site7,site8,site9,site10),parse_dates =['time'], keep_date_col = True, chunksize=chunksize)):
    #We will resample the data to 15 min to be able to plot; dataset is too large and require additional memory to process

    df_chunk = df_chunk.set_index(['time'])
    df_chunk = df_chunk.resample('15 min').mean()
    
    
    # Alternatively, append the chunk to list and merge all
    df_list.append(df_chunk)
    

# Merge all dataframes into one dataframe
data_df = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
data_df.info()

In [ ]:
data_df.shape

In [ ]:
missing_val = (data_df.isna().sum())
print(missing_val)

In [ ]:
%%time
data_df.plot(grid=True, subplots=True, figsize=(18,18))
plt.xlabel('time', fontsize=18)
plt.legend(loc='upper left')
plt.tight_layout()

#### Let's choose regions in Germany located in opposite geographical directions that to say Bremen(North), Herzogenrath(North), Büdingen(South-East) and Schondorf(South) for the project.
#### Notice that Bremen appears to miss data from misses from September 2019 to January 2020.

In [ ]:
#Locations chosen for the project
site2 = "Büdingen"
site3 = "Schondorf"
site4 = "Herzogenrath"
site5 = "Bremen"

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(path2, usecols=("time",site2,site3,site4,site5),parse_dates =['time'], keep_date_col = True, chunksize=chunksize)):
    #We will resample the data to 15 min to be able to plot; dataset is too large and require additional memory to process

    df_chunk = df_chunk.set_index(['time'])
    df_chunk = df_chunk.resample('15 min').mean()
    
    # Append the chunk to list and merge all
    df_list.append(df_chunk)
    

# Merge all dataframes into one dataframe
df_data = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what we have loaded
df_data.info()

In [ ]:
df_data.shape

In [ ]:
# start and end dates of the time series
print ("phase angle dataset: start_date = {}, end_date = {} \n".format(df_data.index.min(), df_data.index.max()))

In [ ]:
missing_val = (df_data.isna().sum())
print(missing_val)

In [ ]:
%%time
df_data.plot(grid=True, subplots=True, figsize=(18,18))
plt.xlabel('time', fontsize=18)
plt.legend(loc='upper left')
plt.tight_layout()

#### Bremen data starts around the beginning of January

In [ ]:
%%time
df_list = [] # list to hold the batch dataframe

for df_chunk in tqdm(pd.read_csv(path2, usecols=("time",site2,site3,site4,site5),parse_dates =['time'], keep_date_col = True, chunksize=chunksize)):
    
    df_chunk = df_chunk.set_index(['time'])  
    # Can process each chunk of dataframe here
    # clean_data(), pre_process_data()
    
    #df_chunk = df_chunk.assign(Büdingen2=df_chunk.Büdingen.fillna(df_chunk.Büdingen.mean()))
    df_chunk = df_chunk.assign(Herzogenrath2=df_chunk.Herzogenrath.fillna(df_chunk.Herzogenrath.mean()))
    df_chunk = df_chunk.assign(Schondorf2=df_chunk.Schondorf.fillna(df_chunk.Schondorf.mean()))
   
    df_chunk['phase_diff1'] = (df_chunk['Bremen'] - df_chunk['Schondorf2'])
    df_chunk['angle1'] = np.sin(np.deg2rad(df_chunk['phase_diff1']))
    
    
    df_chunk['phase_diff2'] = (df_chunk['Herzogenrath2'] - df_chunk['Schondorf2'])
    df_chunk['angle2'] = np.sin(np.deg2rad(df_chunk['phase_diff2']))
    
    df_chunk['phase_diff3'] = (df_chunk['Bremen'] - df_chunk['Büdingen'])
    df_chunk['angle3'] = np.sin(np.deg2rad(df_chunk['phase_diff3']))
    
    #df_chunk['time'] = pd.to_datetime(df_chunk['time'], format='%Y-%m-%d %H:%M:%S')
    df_chunk = df_chunk.resample('15 min').mean()
    #df2_chunk = df2_chunk.set_index(['time'])

    
    # Append the chunk to list and merge all
    df_list.append(df_chunk)

# Merge all dataframes into one dataframe
phasor_data = pd.concat(df_list)

# Delete the dataframe list to release memory
del df_list

# See what I have loaded
phasor_data.info()

In [ ]:
phasor_data.shape

In [ ]:
# start and end dates of the time series
print ("phase angle dataset: start_date = {}, end_date = {} \n".format(phasor_data.index.min(), phasor_data.index.max()))

In [ ]:
missing_val = (phasor_data.isna().sum())
print(missing_val)

In [ ]:
phasor_data.head()

In [ ]:
phasor_data.describe()

In [ ]:
%%time
phasor_data.plot(grid=True, subplots=True, figsize=(18,18))
plt.xlabel('time', fontsize=18)
plt.legend(loc='upper left')
plt.tight_layout()

## Let's use the time series from Jan 2020, since Bremen misses data from September 2019 until Jan 7th, 2020

In [ ]:
angle_data = phasor_data[['angle1', 'angle2', 'angle3']].copy()
#angle_data.head()

phasor = angle_data.copy()
phasor = phasor.reset_index()
time_df = (phasor['time'] >= ('2020-01-07 00:00:00')) & (phasor['time'] <= ('2020-09-30 23:45:00'))
phasor_df = phasor.loc[time_df]
phasor_df = phasor_df.set_index(['time'])

print("angle1 : Bremen_Schondorf")
print("angle2: Herzogenrath_Schondorf")
print("angle3: Bremen_Büdingen")

phasor_df.head()

In [ ]:
phasor_df.isna().sum()

## Visualization of power flow transfer between 2 locations

In [ ]:
fig1 = px.line(x=phasor_df.index, y=phasor_df['angle2'], title='Power flow transfer between Herzogenrath & Schondorf')
fig1.update_xaxes(rangeslider_visible=True)
fig1.show()

In [ ]:
fig2 = px.line(x=phasor_df.index, y=phasor_df['angle1'], title='Power flow transfer between Bremen & Schondorf')
fig2.update_xaxes(rangeslider_visible=True)
fig2.show()

In [ ]:
fig3 = px.line(x=phasor_df.index, y=phasor_df['angle3'], title='Power flow transfer between Bremen & Büdingen')
fig3.update_xaxes(rangeslider_visible=True)
fig3.show()

In [ ]:
# start and end dates of the time series
print ("phase angle dataset: start_date = {}, end_date = {} \n".format(phasor_df.index.min(), phasor_df.index.max()))

## Heatmap visualization between 2 locations

In [ ]:
heatmap_df1 = phasor_df.copy()

#Create variables for Day and hour
heatmap_df1['Day'] = [i.dayofyear for i in heatmap_df1.index]
heatmap_df1['hour'] = [i.hour for i in heatmap_df1.index]

#Group by day and hour and aggregate
heatmap_df1 = heatmap_df1.groupby(['Day','hour']).mean()

#Use unstack function to prepare the data to be plotted
heatmap_df1= heatmap_df1['angle1'].unstack(level=0)

fig1, ax = plt.subplots(figsize=(25, 6))

fig1.canvas.draw()
cmap = 'RdYlBu'


sns.heatmap(heatmap_df1, cmap=cmap)

#plt.title('Herzogenrath_Schondorf \n (Jan 2020 - Sept 2020)', fontweight='bold', fontsize = 14)
plt.title('Bremen_Schondorf', fontweight='bold', fontsize = 14)
plt.xlabel("Day_Number", fontsize=12)
plt.ylabel("Hour of the day", fontsize=12)

plt.yticks(rotation=0)
#plt.xticks('Day -' + from_jan_2020.Day)

labels = [item.get_text() for item in ax.get_xticklabels()]
#labels[2] = 'Day -'

ax.set_xticklabels(labels)
plt.show()

In [ ]:
heatmap_df2 = phasor_df.copy()

#Create variables for Day and hour
heatmap_df2['day'] = [i.dayofyear for i in heatmap_df2.index]
heatmap_df2['hour'] = [i.hour for i in heatmap_df2.index]

#Group by day and hour and aggregate
heatmap_df2 = heatmap_df2.groupby(['day','hour']).mean()

#Use unstack function to prepare the data to be plotted
heatmap_df2= heatmap_df2['angle2'].unstack(level=0)


fig2, ax = plt.subplots(figsize=(25, 6))

fig2.canvas.draw()
cmap = 'RdYlBu'


sns.heatmap(heatmap_df2, cmap=cmap)

#plt.title('Herzogenrath_Schondorf \n (Jan 2020 - Sept 2020)', fontweight='bold', fontsize = 14)
plt.title('Herzogenrath_Schondorf', fontweight='bold', fontsize = 14)
plt.xlabel("Day_Number", fontsize=12)
plt.ylabel("Hour of the day", fontsize=12)

plt.yticks(rotation=0)
#plt.xticks('Day -' + from_jan_2020.Day)

labels = [item.get_text() for item in ax.get_xticklabels()]
#labels[2] = 'Day -'

ax.set_xticklabels(labels)
plt.show()

In [ ]:
heatmap_df3 = phasor_df.copy()

#Create variables for Day and hour
heatmap_df3['Day'] = [i.dayofyear for i in heatmap_df3.index]
heatmap_df3['hour'] = [i.hour for i in heatmap_df3.index]

#Group by day and hour and aggregate
heatmap_df3 = heatmap_df3.groupby(['Day','hour']).mean()


#Use unstack function to prepare the data to be plotted
heatmap_df3= heatmap_df3['angle3'].unstack(level=0)


fig3, ax = plt.subplots(figsize=(25, 6))

fig3.canvas.draw()
#cmap = 'YlGnBu'
#cmap = 'YlGnBu'
#cmap = 'YlOrBr'
#cmap = 'BuGn_r'
cmap = 'RdYlBu'


sns.heatmap(heatmap_df3, cmap=cmap)

#plt.title('Herzogenrath_Schondorf \n (Jan 2020 - Sept 2020)', fontweight='bold', fontsize = 14)
plt.title('Bremen_Büdingen', fontweight='bold', fontsize = 14)
plt.xlabel("Day_Number", fontsize=12)
plt.ylabel("Hour of the day", fontsize=12)

plt.yticks(rotation=0)
#plt.xticks('Day -' + from_jan_2020.Day)

labels = [item.get_text() for item in ax.get_xticklabels()]
#labels[2] = 'Day -'

ax.set_xticklabels(labels)
plt.show()

Frequency Time Series Decomposition

In [ ]:
decompfreq = int(((24*60)/15)*7)
res = sm.tsa.seasonal_decompose(phasor_df.angle1.interpolate(),
                               period=decompfreq,model='additive')
pylab.rcParams['figure.figsize'] = (14, 9)
resplot = res.plot()

## Correlation between solar energy and phasor angle

#### Let's choose time series from Jan 2020 for solar energy as we did for phasor angle data

In [ ]:
aggregate_pv_df.head()

In [ ]:
energy = aggregate_pv_df.copy()
energy = energy.reset_index()
time_series = (energy['time'] >= ('2020-01-07 00:00:00')) & (energy['time'] <= ('2020-09-30 23:45:00'))
pv_df = energy.loc[time_series]
pv_df = pv_df.set_index(['time'])
pv_df.head()

In [ ]:
pv_df.isna().sum()

In [ ]:
# start and end dates of the time series
print ("PV dataset: start_date = {}, end_date = {} \n".format(pv_df.index.min(), pv_df.index.max()))

In [ ]:
data1 = phasor_df.copy()
data2 = pv_df.copy()
data1 = data1.reset_index()
data2 = data2.reset_index()

all_data = data1.merge(data2, left_on=['time'], right_on=['time'], how='right')


#data = data.drop(['Winterthur', 'Dresden','Lleida','Sibiu','Belfort Cedex','Wien (SBA)'], axis = 1)
df = all_data.copy()
df.rename(columns = {'angle1':'phasor_diff1','angle2':'phasor_diff2','angle3':'phasor_diff3',
                   'Hochrechnung_Total':'PV_hoch', 'Prognose_Total':'PV_prog'}, inplace = True)
#df.rename(columns = {'angle1':'phasor_diff1','angle2':'phasor_diff2',
                   #'Hochrechnung_Total':'Solar_energy',
                   #'Prognose_Total':'Prognose'}, inplace = True)
#df.head()
print("phasor_diff1 : Bremen_Schondorf")
print("phasor_diff2: Herzogenrath_Schondorf")
print("phasor_diff3: Bremen_Büdingen")

corr = df.corr().loc[['phasor_diff1', 'phasor_diff2', 'phasor_diff3','PV_hoch', 'PV_prog'], ['phasor_diff1', 'phasor_diff2','phasor_diff3', 'PV_hoch', 'PV_prog']]
display(corr)

#corr = df.corr().loc[['phasor_angle1', 'energy_hoch'], ['phasor_angle1', 'energy_hoch']]
#display(corr)

#fig = plt.subplots(figsize=(7, 4))
#cmap = 'Reds'
#cmap = 'YlGnBu'
#cmap = 'YlOrBr'
#cmap = 'BuGn_r'
#cmap = 'RdYlBu'
#cmap = 'GnBu'
#cmap="OrRd"
#sns.heatmap(corr, cmap=cmap)

#plt.show()


In [ ]:
df.shape

In [ ]:
df.head()

# Forecast

### Looking at solar energy and phasor angle, can you forecast phase angle

In [ ]:
new_df2 = df[['time', 'phasor_diff1', 'PV_hoch']].copy()
new_df2 = new_df2.set_index(['time'])
new_df2.head()

In [ ]:
# start and end dates of the time series
print ("Dataset: start_date = {}, end_date = {} \n".format(new_df2.index.min(), new_df2.index.max()))

In [ ]:
new_df2.describe()

In [ ]:
#the supervised learning algorithm was adopted from
#https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	dff = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(dff.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(dff.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
## Scale all features in range of [0,1].
values = new_df2.values 

#features normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 18)
#print(reframed.head())
# columns we don't need to predict are dropped
#reframed.drop(reframed.columns[[3,5,7,9]], axis=1, inplace=True)
reframed.drop(reframed.columns[[3,5,7,9,11,13,15,17,19,21,23,25,27,29,30,31,33,35,37]], axis=1, inplace=True)
print(reframed.head())

In [ ]:
reframed.info()

In [ ]:
reframed.describe()

In [ ]:
# split into train and test sets
values = reframed.values

train_size = int(len(values) * 0.67)
test_size = len(values) - train_size
train, test = values[0:train_size,:], values[train_size:len(scaled),:]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape) 
# The input is reshaped into a 3D format as expected by LSTMs, namely [samples, timesteps, features].

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu',input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, activation='relu',return_sequences=False))
model.add(Dropout(0.3))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
#model.summary()

# fit model
history = model.fit(train_X, train_y, epochs=20, batch_size=70, validation_data=(test_X, test_y), verbose=1, shuffle=False)


# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Training loss', 'Validation loss'], loc='upper right')

plt.show()

# make a prediction
vhat = model.predict(train_X) ## train
yhat = model.predict(test_X) ## test
test_X = test_X.reshape((test_X.shape[0], 18))
train_X = train_X.reshape((train_X.shape[0], 18))

# invert scaling for forecast (train)
inv_vhat = np.concatenate((vhat, train_X[:, -1:]), axis=1)
inv_vhat = scaler.inverse_transform(inv_vhat)
inv_vhat = inv_vhat[:,0]

# invert scaling for forecast (test)
inv_yhat = np.concatenate((yhat, test_X[:, -1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

# invert scaling for actual (train)
train_y = train_y.reshape((len(train_y), 1))
inv2_y = np.concatenate((train_y, train_X[:, -1:]), axis=1)
inv2_y = scaler.inverse_transform(inv2_y)
inv2_y = inv2_y[:,0]

# invert scaling for actual (test)
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, -1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]


# calculate RMSE and MAE train and test
rmse2 = np.sqrt(mean_squared_error(inv2_y, inv_vhat))
print('Train RMSE: %.3f' % rmse2)
mae2 = mean_absolute_error(inv2_y, inv_vhat)
print('Train MAE: %.3f' % mae2)

rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)
mae = mean_absolute_error(inv_y, inv_yhat)
print('Test MAE: %.3f' % mae)

In [ ]:
## time steps, every step is 15 min (you can easily convert the time step to the actual time index)
## for a demonstration purpose, I only compare the predictions in 50 hours. 

aa=[x for x in range(100)]

plt.plot(aa,inv_y[:100], marker='.', label="Actual data")
plt.plot(aa,inv_yhat[:100], '--r', label="Predicted data")
# plt.tick_params(left=False, labelleft=True) #remove ticks
plt.tight_layout()
sns.despine(top=True)
plt.subplots_adjust(left=0.07)
plt.ylabel('Phasor difference', size=15)
plt.xlabel('Time step', size=15)
plt.legend(fontsize=15)

plt.show()